In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset

from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from readdataUC import UC
import gurobipy as gp
import json
from torch_geometric.data import HeteroData



In [2]:
import copy

class BipartiteGraphConvolution(torch_geometric.nn.MessagePassing):
    """
    The bipartite graph convolution is already provided by pytorch geometric and we merely need
    to provide the exact form of the messages being passed.
    """

    def __init__(self,emb_size):
        super().__init__("add")
            
        self.feature_module_left = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size)
        )
        self.feature_module_edge = torch.nn.Sequential(
            torch.nn.Linear(1, emb_size, bias=False)
        )
        self.feature_module_right = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size, bias=False)
        )
        self.feature_module_final = torch.nn.Sequential(
            torch.nn.LayerNorm(emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )
        self.post_conv_module = torch.nn.Sequential(torch.nn.LayerNorm(emb_size))
        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(2 * emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

    def forward(self, left_features, edge_indices, edge_features, right_features):
        output = self.propagate(
            edge_indices,
            size=(left_features.shape[0], right_features.shape[0]),
            node_features=(left_features, right_features),
            edge_features=edge_features,
        )
        return self.post_conv_module(output)
            
    def message(self, node_features_i, node_features_j, edge_features):
        output = self.feature_module_final(
            self.feature_module_left(node_features_i)
            + self.feature_module_edge(edge_features)
            + self.feature_module_right(node_features_j)
        )
        return output
    
    

class GNNPolicy(torch.nn.Module):
    def __init__(self):
        super(GNNPolicy, self).__init__()
        hidden_channels=64
        out_channels=64
        self.var_node = ['u','p','s']
        self.con_node = ['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
        self.node_types=['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
        self.relations = ['v2c','c2v']
        self.init_sizes=[6,6,6,4,4,4,4,4,4,4]
        edge_feats=1

        self.num_relations=len(self.var_node)*len(self.con_node)
        
        self.vc_idx={}

        self.node_types = ['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']    

        self.convc1 = torch.nn.ModuleList()
        self.concv1 = torch.nn.ModuleList()
        self.convc2 = torch.nn.ModuleList()
        self.concv2 = torch.nn.ModuleList()

        for i in self.var_node:
            for j in self.con_node:
                self.convc1.append(BipartiteGraphConvolution(hidden_channels))
                self.concv1.append(BipartiteGraphConvolution(hidden_channels))
                self.convc2.append(BipartiteGraphConvolution(hidden_channels))
                self.concv2.append(BipartiteGraphConvolution(hidden_channels))



        self.node_embedding = torch.nn.ModuleList()
        for i in range(len(self.node_types)):
            lin = torch.nn.Sequential(
                torch.nn.LayerNorm(self.init_sizes[i]),
                torch.nn.Linear(self.init_sizes[i], hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels,hidden_channels),
                torch.nn.ReLU()
            )
            self.node_embedding.append(lin)
        self.edge_embedding = torch.nn.ModuleList()
        for i in range(self.num_relations):
            lin = torch.nn.LayerNorm(edge_feats)
            self.edge_embedding.append(lin)
        
        self.linv1 = torch.nn.Sequential(
            torch.nn.Linear(2 * hidden_channels, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, hidden_channels),
        )
        self.linc1 = torch.nn.Sequential(
            torch.nn.Linear(2 * hidden_channels, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, hidden_channels),
        )
        self.linv2 = torch.nn.Sequential(
            torch.nn.Linear(2 * out_channels, out_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels, out_channels),
        )
        self.linc2 = torch.nn.Sequential(
            torch.nn.Linear(2 * out_channels, out_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels, out_channels),
        )
        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(out_channels, int(out_channels/2)),
            torch.nn.ReLU(),
            torch.nn.Linear(int(out_channels/2), 1, bias=False)
        )

    def trans_dimensions(self, g):
        data = copy.deepcopy(g)
        for node_type, lin in zip(self.node_types, self.node_embedding):
            data[node_type].x = lin(data[node_type].x)
        cnt=0
        for v in self.var_node:
            for c in self.con_node:
                if 'edge_attr' in data[v,'v2c',c]:
                    data[v,'v2c',c].edge_attr = self.edge_embedding[cnt](data[v,'v2c',c].edge_attr)
                    data[c,'v2c',v].edge_attr = data[v,'v2c',c].edge_attr
                    self.vc_idx[(v,c)]=cnt
                else:
                    self.vc_idx[(v,c)]=-1
                cnt+=1
        return data

    def forward(self, data):
        data = self.trans_dimensions(data)
        for c in self.con_node:
            x = torch.zeros_like(data[c].x)
            for v in self.var_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.convc1[self.vc_idx[(v,c)]](data[v].x, data[v,'v2c',c].edge_index, data[v,'v2c',c].edge_attr,data[c].x)
            data[c].x = self.linc1(torch.cat([data[c].x, x], dim=-1))


        for v in self.var_node:
            x = torch.zeros_like(data[v].x)
            for c in self.con_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.concv1[self.vc_idx[(v,c)]](data[c].x, data[c,'c2v',v].edge_index, data[c,'c2v',v].edge_attr,data[v].x)
            data[v].x = self.linv1(torch.cat([data[v].x, x], dim=-1))

        for c in self.con_node:
            x = torch.zeros_like(data[c].x)
            for v in self.var_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.convc2[self.vc_idx[(v,c)]](data[v].x, data[v,'v2c',c].edge_index, data[v,'v2c',c].edge_attr,data[c].x)
            data[c].x = self.linc2(torch.cat([data[c].x, x], dim=-1))

        for v in self.var_node:
            x = torch.zeros_like(data[v].x)
            for c in self.con_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.concv2[self.vc_idx[(v,c)]](data[c].x, data[c,'c2v',v].edge_index, data[c,'c2v',v].edge_attr,data[v].x)
            data[v].x = self.linv2(torch.cat([data[v].x, x], dim=-1))

        x = self.output_module(data['u'].x).sigmoid()
        return x

In [3]:
#set folder
uc_model = '1bin'
instance = '80_c11_based_8_std'
# instance = '80_c11_based_8_std'
# instance = '5_std'
model = 'demo1'

n=int(instance.split('_')[0])*24


model_save_path = f'../model_save/{uc_model}_{instance}_{model}/'
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [4]:
LEARNING_RATE = 1e-4
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [5]:
train_data = GraphDataset(f'../datasets/{instance}/{uc_model}/second/train/')
valid_data = GraphDataset(f'../datasets/{instance}/{uc_model}/second/valid/')
# train_data = GraphDataset(f'../datasets/mixed/train/')
# valid_data = GraphDataset(f'../datasets/mixed/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [6]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")

载入模型成功


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            presol = predict(batch).squeeze(1)
            loss = loss_fn(presol, weighted_sol)
            
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [8]:
def get_graph(m):
    mvars = m.getVars()
    constrs = m.getConstrs()
    b_vars = torch.tensor([1 if 'u'in v.Varname else 0 for v in mvars])
    coo = m.getA().tocoo()
    # values = coo.data
    # indices = np.vstack((coo.row, coo.col))
    v0 = torch.tensor(m.getAttr("Obj", mvars)).reshape(-1,1)
    v1 = torch.tensor(coo.getnnz(axis=0)).reshape(-1,1)
    v2 = torch.tensor(coo.sum(axis=0)).reshape(-1,1)/v1
    v3 = torch.tensor(coo.max(axis=0).toarray()).reshape(-1,1)
    v4 = torch.tensor(coo.min(axis=0).toarray()).reshape(-1,1)
    v5 = b_vars.clone().reshape(-1,1)
    v_nodes = torch.cat((v0,v1,v2,v3,v4,v5),1).float()
    c0 = torch.tensor(coo.getnnz(axis=1)).reshape(-1,1)
    c1 = torch.tensor(coo.sum(axis=1)).reshape(-1,1)/c0
    c2 = torch.tensor(m.getAttr("RHS", constrs)).reshape(-1,1)
    c3 = torch.tensor([2 if i=='<' else 1 for i in m.getAttr("Sense", constrs)]).reshape(-1,1)
    c_nodes = torch.cat((c0,c1,c2,c3),1).float()

    cons=['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']

    graph = HeteroData()

    n=len(mvars)
    T=24
    N=int(sum(b_vars)/T)

    if N*T == n/3:
        
        graph['u'].x = v_nodes[:N*T]
        graph['p'].x = v_nodes[N*T:2*N*T]
        graph['s'].x = v_nodes[2*N*T:3*N*T]

        r = 0
        idx=0
        for l, c in enumerate(constrs):
            if cons[idx] in c.ConstrName and l != len(constrs) - 1:
                continue
            if l == len(constrs) - 1:
                l+=1
            graph[cons[idx]].x = c_nodes[r:l]
            uindice = []
            pindice = []
            sindice = []
            uvalues = []
            pvalues = []
            svalues = []
            for constr, variable, value in zip(coo.row, coo.col, coo.data):
                if r <= constr < l:
                    if variable < N*T:
                        uindice.append((variable, constr-r))
                        uvalues.append(value)
                    elif N*T <= variable < 2*N*T:
                        pindice.append((variable-N*T, constr-r))
                        pvalues.append(value)
                    elif variable >= 2*N*T:
                        sindice.append((variable-2*N*T, constr-r))
                        svalues.append(value)

            uindice = np.array(uindice).T
            pindice = np.array(pindice).T
            sindice = np.array(sindice).T
            uvalues = torch.tensor(uvalues, dtype=torch.float32).reshape(-1,1)
            pvalues = torch.tensor(pvalues, dtype=torch.float32).reshape(-1,1)
            svalues = torch.tensor(svalues, dtype=torch.float32).reshape(-1,1)

            if uindice.size:
                graph['u','v2c',cons[idx]].edge_index = torch.from_numpy(uindice)
                graph['u','v2c',cons[idx]].edge_attr = uvalues
                graph[cons[idx],'c2v','u'].edge_index = torch.from_numpy(uindice[[1,0]])
                graph[cons[idx],'c2v','u'].edge_attr = uvalues
            if pindice.size:
                graph['p','v2c',cons[idx]].edge_index = torch.from_numpy(pindice)
                graph['p','v2c',cons[idx]].edge_attr = pvalues
                graph[cons[idx],'c2v','p'].edge_index = torch.from_numpy(pindice[[1,0]])
                graph[cons[idx],'c2v','p'].edge_attr = pvalues
            if sindice.size:                
                graph['s','v2c',cons[idx]].edge_index = torch.from_numpy(sindice)
                graph['s','v2c',cons[idx]].edge_attr = svalues
                graph[cons[idx],'c2v','s'].edge_index = torch.from_numpy(sindice[[1,0]])
                graph[cons[idx],'c2v','s'].edge_attr = svalues
            r = l
            idx += 1
        graph
        return graph


In [9]:
def ps_accelerate(sol,file):
    
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.setParam('MIPGap', 1e-3)
        env.start()
        m = gp.read(file,env)
        f = m.getObjective()
        k=1
        
        sol = he_fs(m,sol,k,instance)
        sol = remove_initial(sol,m)
        day = int(file.split('/')[-1].split('_')[0])
        sol = restore_on_off(sol,instance,day)
        m = fix_variable(m,sol)

        m.Params.TimeLimit = 10
        m.optimize()
        if m.Status==2 or m.Status ==9:
            return f.getValue()
        else:
            return -1

In [10]:
def valid(predict):
    lp_path=f'../datasets/{instance}/{uc_model}/lp/valid/'
    bin3_path=f'../datasets/{instance}/3bin/lp/valid/'
    with open(f'../datasets/{instance}/{uc_model}/lp/valid/obj.json', 'r') as f:
        objs = json.load(f)
    inf_cnt=0
    opt_loss=[]
    files = os.listdir(lp_path)
    files = [file for file in files if file.endswith('.lp')][::20]
    for file in files:
        lp_file = os.path.join(lp_path, file)
        bin3_file = os.path.join(bin3_path, file)
        with gp.Env(empty=True) as env:
            env.setParam('OutputFlag', 0)
            env.setParam('MIPGap', 1e-3)
            env.start()
            m = gp.read(lp_file,env)
        graph=get_graph(m).to(DEVICE)
        sol = predict(graph)
        # sol = torch.rand(n,1)
        # pt_path = f'../datasets/{instance}/1bin/valid/'
        # sol = torch.load(pt_path+file.split('.')[0]+'.pt')['opt_sol']
        obj = ps_accelerate(sol,bin3_file)
        if obj == -1:
            inf_cnt+=1
        else:
            opt_loss.append((obj-objs[file.split('.')[0]])/objs[file.split('.')[0]])
    opt_loss=np.array(opt_loss)
    if len(files)==0:
        print('no file')
    return opt_loss.mean(),inf_cnt/(len(files))

In [11]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
best_val_loss = 0.0754445397727926
best_acc=1

In [13]:
cur_epoch = 0
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f}")
    acc,infs=valid(PredictModel)
    print("acc",acc,'inf',infs)
    # if acc<best_acc:
    #     cur_epoch=0
    #     best_acc = acc
    #     print(best_acc)
    #     torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.076820	Epoch 0 Valid loss: 0.078766
acc 0.011619961385531124 inf 0.0
Epoch 1 Train loss: 0.076179	Epoch 1 Valid loss: 0.076357
acc 0.010769539148785881 inf 0.0
Epoch 2 Train loss: 0.078461	Epoch 2 Valid loss: 0.080233
acc 0.010242905096399984 inf 0.0
Epoch 3 Train loss: 0.076145	Epoch 3 Valid loss: 0.078833
acc 0.009617515156231242 inf 0.0
Epoch 4 Train loss: 0.076840	Epoch 4 Valid loss: 0.076418
acc 0.01002629856226573 inf 0.0
Epoch 5 Train loss: 0.076589	Epoch 5 Valid loss: 0.077202
acc 0.010473934931254791 inf 0.0
Epoch 6 Train loss: 0.076137	Epoch 6 Valid loss: 0.077234
acc 0.010992032691070297 inf 0.0
Epoch 7 Train loss: 0.075893	Epoch 7 Valid loss: 0.076087
acc 0.01068584712829466 inf 0.0
Epoch 8 Train loss: 0.076700	Epoch 8 Valid loss: 0.077057
acc 0.011161853739172558 inf 0.0
Epoch 9 Train loss: 0.076630	Epoch 9 Valid loss: 0.077164
acc 0.010011028174339291 inf 0.02702702702702703
Epoch 10 Train loss: 0.076179	Epoch 10 Valid loss: 0.076097
acc 0.0104988313

: 